In [1]:
import os
os.chdir('../..')

In [215]:
import numpy as np
import pandas as pd
from collections import Iterable, defaultdict
import random
from os import listdir
from os.path import isfile, join
import json

from itertools import combinations

import matplotlib.pyplot as plt

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Functions

In [110]:
def is_a_DATGAN(name):
    if 'TGAN' in name or 'CTGAN' in name:
        return False
    else:
        return True

def compute_stats(freq_list_orig, freq_list_synth):
    """
    Different statistics computed on the frequency list
    
    """
    freq_list_orig, freq_list_synth = np.array(freq_list_orig), np.array(freq_list_synth)
    corr_mat = np.corrcoef(freq_list_orig, freq_list_synth)
    corr = corr_mat[0, 1]
    if np.isnan(corr): corr = 0.0
    # MAE
    mae = np.absolute(freq_list_orig - freq_list_synth).mean()
    # RMSE
    rmse = np.linalg.norm(freq_list_orig - freq_list_synth) / np.sqrt(len(freq_list_orig))
    # SRMSE
    freq_list_orig_avg = freq_list_orig.mean()
    srmse = rmse / freq_list_orig_avg
    # r-square
    u = np.sum((freq_list_synth - freq_list_orig)**2)
    v = np.sum((freq_list_orig - freq_list_orig_avg)**2)
    r2 = 1.0 - u / v
    stat = {'mae': mae, 'rmse': rmse, 'r2': r2, 'srmse': srmse, 'corr': corr}
    
    return stat

# Get all models and associated files

In [4]:
dataset = 'Chicago'
n_models = 5
n_data = 5

models = ['CTGAN', 'TGAN']

for i in ['WGAN', 'SGAN', 'WGGP']:
    for j in ['WI', 'OR', 'WO']:
        for k in ['NO', 'BO', 'OD']:
            models.append('{}_{}_{}'.format(i,j,k))
            
models.sort()

files_ = {}

for m in models:
    tmp = []
    if is_a_DATGAN(m):
        spl = m.split('_')
        for i in range(n_models):
            for j in range(n_data):
                tmp.append('{}_{}_{:0>2}_{}_{:0>2}.csv'.format(spl[0], spl[1], i+1,  spl[2], j+1))
    else:
        for i in range(n_models):
            for j in range(n_data):
                tmp.append('{}_{:0>2}_{:0>2}.csv'.format(m, i+1, j+1))
    files_[m] = tmp


input_folder = '../synth_data/{}/'.format(dataset)

In [102]:
df_orig = pd.read_csv('../data/' + dataset + '/data.csv')

In [103]:
if dataset is 'Chicago':
    continuous_cols = ['distance', 'age', 'departure_time']
elif dataset is 'LPMC':
    continuous_cols = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [104]:
bins_cont = {}

for c in continuous_cols:
    #bins_cont[c] = pd.qcut(df_orig[c], q=10, retbins=True)[1]
    bins_cont[c] = pd.cut(df_orig[c], bins=10, retbins=True)[1]
    bins_cont[c][0] = -np.inf
    bins_cont[c][-1] = np.inf
    df_orig[c] = pd.cut(df_orig[c], bins=bins_cont[c])

In [105]:
df_orig.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,"(-inf, 6.971]",2,3,3,detached,6,0,"(29.4, 39.2]",1,4,PTE,"(19.093, 21.48]"
1,drive,2,SHOPPING,"(-inf, 6.971]",3,3,3,detached,7,0,"(49.0, 58.8]",1,5,FTE,"(16.707, 19.093]"
2,drive,2,SHOPPING,"(-inf, 6.971]",1,1,0,detached,3,0,"(78.4, 88.2]",1,3,PTE,"(7.16, 9.547]"
3,drive,2,OTHER,"(-inf, 6.971]",2,2,0,detached,5,1,"(39.2, 49.0]",1,5,FTE,"(11.933, 14.32]"
4,passenger,1,SHOPPING,"(-inf, 6.971]",2,2,1,detached,4,0,"(29.4, 39.2]",0,3,Unemployed,"(9.547, 11.933]"


In [106]:
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']
orig_str = 'random-original'

# Stats per individual column

In [201]:
all_stats = {}

# Go through each model
for i, m in enumerate(models):
    
    print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))
    
    all_stats[m] = {}
    
    for c in df_orig.columns:
        all_stats[m][c] = {}
        for s in stats_str:
            all_stats[m][c][s] = []
    
    # Load all dataframes for current model
    dfs = [pd.read_csv(input_folder + f) for f in files_[m]]
    
    # Go through all dataframes generated for each model
    for df in dfs:
        
        # Discretize continuous columns
        for c in continuous_cols:
            df[c] = pd.cut(df[c], bins=bins_cont[c])
        
        # Go through each columns
        for c in df_orig.columns:
            
            agg_vars = [c]
            
            real = df_orig.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)
            
            synth = df.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)
            
            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)
            
            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])
            
            for s in sts:
                all_stats[m][c][s].append(sts[s])

Preparing stats for model CTGAN (1/29)
Preparing stats for model SGAN_OR_BO (2/29)
Preparing stats for model SGAN_OR_NO (3/29)
Preparing stats for model SGAN_OR_OD (4/29)
Preparing stats for model SGAN_WI_BO (5/29)
Preparing stats for model SGAN_WI_NO (6/29)
Preparing stats for model SGAN_WI_OD (7/29)
Preparing stats for model SGAN_WO_BO (8/29)
Preparing stats for model SGAN_WO_NO (9/29)
Preparing stats for model SGAN_WO_OD (10/29)
Preparing stats for model TGAN (11/29)
Preparing stats for model WGAN_OR_BO (12/29)
Preparing stats for model WGAN_OR_NO (13/29)
Preparing stats for model WGAN_OR_OD (14/29)
Preparing stats for model WGAN_WI_BO (15/29)
Preparing stats for model WGAN_WI_NO (16/29)
Preparing stats for model WGAN_WI_OD (17/29)
Preparing stats for model WGAN_WO_BO (18/29)
Preparing stats for model WGAN_WO_NO (19/29)
Preparing stats for model WGAN_WO_OD (20/29)
Preparing stats for model WGGP_OR_BO (21/29)
Preparing stats for model WGGP_OR_NO (22/29)
Preparing stats for model WGGP

In [202]:
stats_orig = {}

for c in df_orig.columns:
    stats_orig[c] = {}
    for s in stats_str:
        stats_orig[c][s] = []

for i in range(n_models*n_data):

    train = df_orig.sample(int(len(df_orig) * 0.5))
    train.index = range(len(train))
    test = df_orig[~df_orig.index.isin(train.index)]
    test.index = range(len(test))

    # Go through each columns
    for c in df_orig.columns:

        agg_vars = [c]

        real = train.copy()
        real['count'] = 1
        real = real.groupby(agg_vars, observed=True).count()
        real /= len(df_orig)

        synth = test.copy()
        synth['count'] = 1
        synth = synth.groupby(agg_vars, observed=True).count()
        synth /= len(df)

        real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
        real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

        sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

        for s in sts:
            stats_orig[c][s].append(sts[s])

In [203]:
all_stats[orig_str] = stats_orig

In [204]:
with open('./notebooks/results/single_columns.json', 'w') as outfile:
    outfile.write(json.dumps(all_stats))

In [205]:
with open('./notebooks/results/single_columns.json', 'r') as infile:
    all_stats = json.loads(infile.read())

In [206]:
res = {}

for test in ['all', 'cont', 'cat']:
    
    res[test] = {}
    
    if test == 'all':
        cols = df_orig.columns
    elif test == 'cont':
        cols = continuous_cols
    elif test == 'cat':
        cols = set(df_orig.columns) - set(continuous_cols)

    for s in stats:
        res[test][s] = {}

    for m in all_stats.keys():

        for s in stats:
            res[test][s][m] = []

            for i in range(n_models*n_data):
                tmp = []

                for c in cols:
                    tmp.append(all_stats[m][c][s][i])

                res[test][s][m].append(np.mean(tmp))

In [207]:
avg = {}

for test in ['all', 'cont', 'cat']:
    
    avg[test] = {}

    for s in stats:
        avg[test][s] = {}

        for m in all_stats.keys():
            avg[test][s][m] = {
                'mean': np.mean(res[test][s][m]),
                'std': np.std(res[test][s][m])
            }

In [208]:
for test in ['all', 'cont', 'cat']:
    
    if test == 'all':
        str_ = 'on all columns'
    elif test == 'cont':
        str_ = 'on continuous columns'
    elif test == 'cat':
        str_ = 'on categorical columns'
        
    for s in ['srmse']:#stats:
        print('Ranking {} based on {}:'.format(str_, s.upper()))

        if s in ['r2', 'corr']:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])[::-1]}
        else:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])}

        for i, item in enumerate(sorted_dct):
            print('  {:>2}. {:<20} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
        print()


Ranking on all columns based on SRMSE:
   1. random-original      - 5.38e-02 ± 3.13e-03
   2. WGAN_WI_NO           - 6.31e-02 ± 6.91e-03
   3. WGAN_WO_NO           - 6.71e-02 ± 8.64e-03
   4. SGAN_WO_NO           - 6.72e-02 ± 5.85e-03
   5. SGAN_WI_NO           - 7.09e-02 ± 1.08e-02
   6. SGAN_WO_OD           - 7.15e-02 ± 6.20e-03
   7. SGAN_WI_OD           - 7.25e-02 ± 1.02e-02
   8. SGAN_WO_BO           - 7.37e-02 ± 5.87e-03
   9. WGAN_OR_BO           - 7.54e-02 ± 7.72e-03
  10. SGAN_WI_BO           - 7.58e-02 ± 1.11e-02
  11. WGAN_OR_OD           - 7.60e-02 ± 6.86e-03
  12. WGAN_WI_BO           - 7.68e-02 ± 8.90e-03
  13. SGAN_OR_OD           - 7.80e-02 ± 6.31e-03
  14. WGAN_WI_OD           - 7.98e-02 ± 1.07e-02
  15. SGAN_OR_BO           - 8.11e-02 ± 5.76e-03
  16. WGGP_WO_NO           - 9.31e-02 ± 1.86e-02
  17. WGAN_WO_OD           - 1.00e-01 ± 1.13e-02
  18. WGAN_WO_BO           - 1.02e-01 ± 1.47e-02
  19. WGGP_WO_OD           - 1.03e-01 ± 2.05e-02
  20. WGGP_WO_BO           - 1

# Stats per couple columns

In [223]:
combs = []

for k in combinations(df_orig.columns, 2):
    combs.append(k[0] + '::' + k[1])

In [178]:
all_stats = {}

# Go through each model
for i, m in enumerate(models):
    
    print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))
    
    all_stats[m] = {}
    
    for c in combs:
        all_stats[m][c] = {}
        for s in stats_str:
            all_stats[m][c][s] = []
    
    # Load all dataframes for current model
    dfs = [pd.read_csv(input_folder + f) for f in files_[m]]
    
    # Go through all dataframes generated for each model
    for df in dfs:
        
        # Discretize continuous columns
        for c in continuous_cols:
            df[c] = pd.cut(df[c], bins=bins_cont[c])
        
        # Go through each columns
        for c in combs:
            
            agg_vars = c.split('::')

            real = df_orig.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = df.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                all_stats[m][c][s].append(sts[s])

Preparing stats for model CTGAN (1/29)
Preparing stats for model SGAN_OR_BO (2/29)
Preparing stats for model SGAN_OR_NO (3/29)
Preparing stats for model SGAN_OR_OD (4/29)
Preparing stats for model SGAN_WI_BO (5/29)
Preparing stats for model SGAN_WI_NO (6/29)
Preparing stats for model SGAN_WI_OD (7/29)
Preparing stats for model SGAN_WO_BO (8/29)
Preparing stats for model SGAN_WO_NO (9/29)
Preparing stats for model SGAN_WO_OD (10/29)
Preparing stats for model TGAN (11/29)
Preparing stats for model WGAN_OR_BO (12/29)
Preparing stats for model WGAN_OR_NO (13/29)
Preparing stats for model WGAN_OR_OD (14/29)
Preparing stats for model WGAN_WI_BO (15/29)
Preparing stats for model WGAN_WI_NO (16/29)
Preparing stats for model WGAN_WI_OD (17/29)
Preparing stats for model WGAN_WO_BO (18/29)
Preparing stats for model WGAN_WO_NO (19/29)
Preparing stats for model WGAN_WO_OD (20/29)
Preparing stats for model WGGP_OR_BO (21/29)
Preparing stats for model WGGP_OR_NO (22/29)
Preparing stats for model WGGP

In [191]:
stats_orig = {}

for c in combs:
    stats_orig[c] = {}
    for s in stats_str:
        stats_orig[c][s] = []

for i in range(n_models*n_data):

    train = df_orig.sample(int(len(df_orig) * 0.5))
    train.index = range(len(train))
    test = df_orig[~df_orig.index.isin(train.index)]
    test.index = range(len(test))

    # Go through each columns
    for c in combs:

        agg_vars = c.split('::')

        real = train.copy()
        real['count'] = 1
        real = real.groupby(agg_vars, observed=True).count()
        real /= len(df_orig)

        synth = test.copy()
        synth['count'] = 1
        synth = synth.groupby(agg_vars, observed=True).count()
        synth /= len(df)

        real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
        real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

        sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

        for s in sts:
            stats_orig[c][s].append(sts[s])

In [192]:
all_stats[orig_str] = stats_orig

In [193]:
with open('./notebooks/results/couple_combinations.json', 'w') as outfile:
    outfile.write(json.dumps(all_stats))

In [196]:
with open('./notebooks/results/couple_combinations.json', 'r') as infile:
    all_stats = json.loads(infile.read())

In [197]:
res = {}

for s in stats:
    res[s] = {}

for m in all_stats.keys():

    for s in stats:
        res[s][m] = []

        for i in range(n_models*n_data):
            tmp = []

            for c in combs:
                tmp.append(all_stats[m][c][s][i])

            res[s][m].append(np.mean(tmp))

In [198]:
avg = {}

for s in stats:
    avg[s] = {}

    for m in all_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [199]:
for s in ['srmse']:#stats:
    print('Ranking on all coupled combinations based on {}:'.format(s.upper()))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<20} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()


Ranking on all coupled combinations based on SRMSE:
   1. random-original      - 1.26e-01 ± 7.43e-03
   2. WGAN_WI_NO           - 1.79e-01 ± 1.17e-02
   3. WGAN_WI_BO           - 1.93e-01 ± 1.68e-02
   4. SGAN_WO_NO           - 1.96e-01 ± 1.12e-02
   5. WGAN_WI_OD           - 2.01e-01 ± 1.74e-02
   6. SGAN_WO_OD           - 2.02e-01 ± 1.30e-02
   7. WGAN_OR_BO           - 2.04e-01 ± 1.25e-02
   8. WGAN_OR_OD           - 2.04e-01 ± 1.08e-02
   9. SGAN_WI_NO           - 2.08e-01 ± 1.75e-02
  10. SGAN_WO_BO           - 2.09e-01 ± 1.64e-02
  11. SGAN_WI_OD           - 2.09e-01 ± 1.67e-02
  12. SGAN_OR_OD           - 2.10e-01 ± 7.24e-03
  13. SGAN_OR_BO           - 2.17e-01 ± 5.73e-03
  14. SGAN_WI_BO           - 2.19e-01 ± 1.94e-02
  15. WGAN_WO_NO           - 2.21e-01 ± 2.82e-02
  16. WGGP_WO_NO           - 2.50e-01 ± 5.13e-02
  17. WGAN_WO_OD           - 2.57e-01 ± 3.81e-02
  18. WGGP_WO_OD           - 2.61e-01 ± 5.15e-02
  19. WGAN_WO_BO           - 2.62e-01 ± 4.57e-02
  20. WGGP_WO_BO 

# Stats per trouple columns

In [225]:
combs = []

for k in combinations(df_orig.columns, 3):
    combs.append(k[0] + '::' + k[1] + '::' + k[2])

In [228]:
all_stats = {}

# Go through each model
for i, m in enumerate(models):
    
    print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))
    
    all_stats[m] = {}
    
    for c in combs:
        all_stats[m][c] = {}
        for s in stats_str:
            all_stats[m][c][s] = []
    
    # Load all dataframes for current model
    dfs = [pd.read_csv(input_folder + f) for f in files_[m]]
    
    # Go through all dataframes generated for each model
    for df in dfs:
        
        # Discretize continuous columns
        for c in continuous_cols:
            df[c] = pd.cut(df[c], bins=bins_cont[c])
        
        # Go through each columns
        for c in combs:
            
            agg_vars = c.split('::')

            real = df_orig.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = df.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                all_stats[m][c][s].append(sts[s])

Preparing stats for model CTGAN (1/29)
Preparing stats for model SGAN_OR_BO (2/29)
Preparing stats for model SGAN_OR_NO (3/29)
Preparing stats for model SGAN_OR_OD (4/29)
Preparing stats for model SGAN_WI_BO (5/29)
Preparing stats for model SGAN_WI_NO (6/29)
Preparing stats for model SGAN_WI_OD (7/29)
Preparing stats for model SGAN_WO_BO (8/29)
Preparing stats for model SGAN_WO_NO (9/29)
Preparing stats for model SGAN_WO_OD (10/29)
Preparing stats for model TGAN (11/29)
Preparing stats for model WGAN_OR_BO (12/29)
Preparing stats for model WGAN_OR_NO (13/29)
Preparing stats for model WGAN_OR_OD (14/29)
Preparing stats for model WGAN_WI_BO (15/29)
Preparing stats for model WGAN_WI_NO (16/29)
Preparing stats for model WGAN_WI_OD (17/29)
Preparing stats for model WGAN_WO_BO (18/29)
Preparing stats for model WGAN_WO_NO (19/29)
Preparing stats for model WGAN_WO_OD (20/29)
Preparing stats for model WGGP_OR_BO (21/29)
Preparing stats for model WGGP_OR_NO (22/29)
Preparing stats for model WGGP

In [229]:
stats_orig = {}

for c in combs:
    stats_orig[c] = {}
    for s in stats_str:
        stats_orig[c][s] = []

for i in range(n_models*n_data):

    train = df_orig.sample(int(len(df_orig) * 0.5))
    train.index = range(len(train))
    test = df_orig[~df_orig.index.isin(train.index)]
    test.index = range(len(test))

    # Go through each columns
    for c in combs:

        agg_vars = c.split('::')

        real = train.copy()
        real['count'] = 1
        real = real.groupby(agg_vars, observed=True).count()
        real /= len(df_orig)

        synth = test.copy()
        synth['count'] = 1
        synth = synth.groupby(agg_vars, observed=True).count()
        synth /= len(df)

        real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
        real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

        sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

        for s in sts:
            stats_orig[c][s].append(sts[s])

In [230]:
all_stats[orig_str] = stats_orig

In [231]:
with open('./notebooks/results/trouple_combinations.json', 'w') as outfile:
    outfile.write(json.dumps(all_stats))

In [232]:
with open('./notebooks/results/trouple_combinations.json', 'r') as infile:
    all_stats = json.loads(infile.read())

In [233]:
res = {}

for s in stats:
    res[s] = {}

for m in all_stats.keys():

    for s in stats:
        res[s][m] = []

        for i in range(n_models*n_data):
            tmp = []

            for c in combs:
                tmp.append(all_stats[m][c][s][i])

            res[s][m].append(np.mean(tmp))

In [234]:
avg = {}

for s in stats:
    avg[s] = {}

    for m in all_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [237]:
for s in ['srmse']:#stats:
    print('Ranking on all triple combinations based on {}:'.format(s.upper()))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<20} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()


Ranking on all triple combinations based on SRMSE:
   1. random-original      - 2.42e-01 ± 6.47e-03
   2. WGAN_WI_NO           - 3.65e-01 ± 1.84e-02
   3. WGAN_WI_BO           - 3.70e-01 ± 2.52e-02
   4. WGAN_WI_OD           - 3.80e-01 ± 2.27e-02
   5. SGAN_WO_NO           - 4.01e-01 ± 2.07e-02
   6. WGAN_OR_OD           - 4.04e-01 ± 1.47e-02
   7. WGAN_OR_BO           - 4.05e-01 ± 1.73e-02
   8. SGAN_WO_OD           - 4.09e-01 ± 2.47e-02
   9. SGAN_OR_OD           - 4.15e-01 ± 1.44e-02
  10. SGAN_WO_BO           - 4.21e-01 ± 3.11e-02
  11. SGAN_WI_NO           - 4.23e-01 ± 2.54e-02
  12. SGAN_WI_OD           - 4.24e-01 ± 2.34e-02
  13. SGAN_OR_BO           - 4.28e-01 ± 1.58e-02
  14. SGAN_WI_BO           - 4.40e-01 ± 2.65e-02
  15. WGAN_WO_NO           - 4.62e-01 ± 5.41e-02
  16. WGGP_WO_NO           - 4.81e-01 ± 9.03e-02
  17. WGGP_WO_OD           - 4.83e-01 ± 8.72e-02
  18. WGAN_WO_OD           - 4.83e-01 ± 7.46e-02
  19. WGAN_WO_BO           - 4.92e-01 ± 8.49e-02
  20. WGGP_WO_BO  